In [ ]:
import json

In [ ]:
def txt_to_dzn(path):
    print(path)
    name = path.split(".")[0]
    with open(path) as f:
        data = json.load(f)
    
    
    alphabet = "{"
    for char in data["alphabet"]:
        alphabet += f"{char},"
    alphabet = alphabet[:-1]
    alphabet += "}"
    alphabet = f"Alphabet = {alphabet};\n"
    # print(alphabet)
    
    num_strings = data["num_strings"]
    ns = f"ns = {num_strings};\n"
    # print(ns)
    
    max_length = max(data["str_length"])
    nc = f"nc = {max_length};\n"
    # print(nc)
    
    neighbours = "[| "
    for s in data["strings"]:
        for i in range(max(max_length, len(s))):
            if i < len(s):
                neighbours += f"I({s[i]}),"
            else:
                neighbours += f"Null,"
        neighbours = neighbours[:-1]
        neighbours += " | "
    neighbours = "neighbours = " + neighbours[:-1] + "];\n"
    # print(neighbours)
    
    with open(f'{name}.dzn', 'w') as f_new:
        f_new.write(alphabet)
        f_new.write(ns)
        f_new.write(nc)
        f_new.write(neighbours)

In [ ]:
txt_to_dzn("test.txt")

In [ ]:
import os
for root, dirs, files in os.walk("instances"):
    for file in files:
        if file.endswith(".txt"):
            path = os.path.join(root, file)
            txt_to_dzn(path)

In [1]:
import subprocess
import time
import pandas as pd
import os

In [13]:
results = pd.DataFrame(columns=["file", "best", "score", "time"])
timelimit = 60 * 1000

for root, dirs, files in os.walk("instances"):
    for file in sorted(files):
        if file.endswith(".dzn"):
            print(root + "/" +  file)
            t0 = time.time()
            out = subprocess.check_output([
                "minizinc",
                "Minimum_editing_distance.mzn",
                root + "/" +  file,
                "--all-solutions",
                "--time-limit", str(timelimit)
            ])
            
            t1 = time.time()
                  
            parsed_out = str(out).split("Results: {")[-1].split("}")[0].split(" ") # get last solution
            best = parsed_out[1]
            score = int(parsed_out[3])
            print(f"Found {best} with score {score} in {t1-t0:.2f}s")
                  
            results = results.append({"file": file, "best": best, "score": score, "time": t1-t0}, ignore_index=True)           

instances/p2/p2_10_13-0.dzn
Found AAGTCA with score 53 in 42.12s
instances/p2/p2_10_13-1.dzn
Found CAATAGTCTGAAA with score 91 in 60.20s
instances/p2/p2_10_13-2.dzn
Found GCGTACCATGCAA with score 82 in 60.23s
instances/p2/p2_10_13-3.dzn
Found ATCGTCAACTGCA with score 85 in 60.20s
instances/p2/p2_10_13-4.dzn
Found AGCACATAGTCAA with score 80 in 60.21s
instances/p2/p2_10_20-0.dzn
Found CTAGCTGGATCAAAA with score 107 in 60.26s
instances/p2/p2_10_20-1.dzn
Found CCTGAAGATCCAAAAAAA with score 141 in 60.37s
instances/p2/p2_10_20-2.dzn
Found CTGGCATCGGAAAAAAAAA with score 162 in 60.45s
instances/p2/p2_10_20-3.dzn
Found TGCGATACTGCAAAAAAAA with score 149 in 60.42s
instances/p2/p2_10_20-4.dzn
Found GTACTGCGTCGAAAAAA with score 131 in 60.28s
instances/p2/p2_10_3-0.dzn
Found  with score 15 in 0.13s
instances/p2/p2_10_3-1.dzn
Found CA with score 14 in 0.11s
instances/p2/p2_10_3-2.dzn
Found G with score 16 in 0.14s
instances/p2/p2_10_3-3.dzn
Found G with score 17 in 0.13s
instances/p2/p2_10_3-4.dzn


Found ACCAAACAAAAACCAAAAA with score 70 in 60.76s
instances/p1/p1_10_20-1.dzn
Found CAAAACACACAACAAAAAAA with score 96 in 60.76s
instances/p1/p1_10_20-2.dzn
Found ACAAACAACAAAACACAAAA with score 64 in 60.75s
instances/p1/p1_10_20-3.dzn
Found AACACACAAACAAAAAAACA with score 78 in 60.76s
instances/p1/p1_10_20-4.dzn
Found AAAACAACACAAAAACAAAA with score 72 in 60.76s
instances/p1/p1_10_3-0.dzn
Found AAA with score 8 in 0.11s
instances/p1/p1_10_3-1.dzn
Found AAA with score 14 in 0.12s
instances/p1/p1_10_3-2.dzn
Found AA with score 12 in 0.13s
instances/p1/p1_10_3-3.dzn
Found AAA with score 16 in 0.11s
instances/p1/p1_10_3-4.dzn
Found AA with score 16 in 0.13s
instances/p1/p1_10_5-0.dzn
Found AAAA with score 22 in 0.23s
instances/p1/p1_10_5-1.dzn
Found ACAAA with score 22 in 0.25s
instances/p1/p1_10_5-2.dzn
Found ACAAA with score 20 in 0.25s
instances/p1/p1_10_5-3.dzn
Found AAAA with score 20 in 0.25s
instances/p1/p1_10_5-4.dzn
Found AAAA with score 22 in 0.24s
instances/p1/p1_10_8-0.dzn
Fou

In [14]:
results

,file,best,score,time
0,p2_10_13-0.dzn,AAGTCA,53,42.116037
1,p2_10_13-1.dzn,CAATAGTCTGAAA,91,60.202126
2,p2_10_13-2.dzn,GCGTACCATGCAA,82,60.229660
3,p2_10_13-3.dzn,ATCGTCAACTGCA,85,60.204492
4,p2_10_13-4.dzn,AGCACATAGTCAA,80,60.214815
...,...,...,...,...
245,p1_6_8-0.dzn,CAAACAAA,18,0.374833
246,p1_6_8-1.dzn,ACAACCAA,16,0.421805
247,p1_6_8-2.dzn,AACCAAAA,28,0.401498
248,p1_6_8-3.dzn,CAAACAAA,18,0.364164


In [15]:
results.to_csv("results_1_minute_limit.csv")

In [8]:
t0 = time.time()
out = subprocess.check_output(["minizinc", "Minimum_editing_distance.mzn", "instances/p1/p1_10_8-4.dzn", "--all-solutions", "--time-limit", "100"])
t1 = time.time()
print(t1 - t0, out)

0.13161110877990723 b'=====UNKNOWN=====\n'


In [5]:
parsed_out = str(out).split("Results: {")[-1].split("}")[0].split(" ")
parsed_out

['median_string:', 'AACACAAA', 'global_editing_distance:', '38']